In [1]:
!nvidia-smi -L
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

GPU 0: NVIDIA A100-SXM4-80GB (UUID: GPU-e4686931-f447-94e1-8676-4dfcfe3ed155)
PyTorch version: 2.9.0+cu126
CUDA available: True
CUDA device: NVIDIA A100-SXM4-80GB


In [2]:
# Install required packages
!pip install -q torch torchvision matplotlib opencv-python-headless pyyaml tensorboard tqdm scikit-image lpips kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.4 MB/s eta 0:00:00


In [3]:
from google.colab import drive, files
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cd /content/drive/MyDrive/research/locpaint

/content/drive/MyDrive/research/locpaint


In [5]:
ls

baseline/                             evaluation/
compare_models.py                     localizer/
context_encoder_localizer_demo.ipynb  staged/
evaluate_models.py                    test_discriminator_detection.py
evaluate.py                           utils/


In [ ]:
!python localizer/train_localizer.py \
    --d_capacity_factor 0.5 \
    --d_num_layers 2 \
    --mask_type random_square \
    --num_epochs 30 \
    --dataset cifar10 \
    --viz_freq 250


Starting Localizer Training from epoch 0
Dataset: cifar10, Batches: 782
Device: cuda
Mask: random_square (64x64)

Discriminator Settings:
  Capacity factor: 0.5
  Dropout rate: 0
  Num layers: 2
  Update ratio: 1/2
  Learning rate: 0.0002 (Generator: 0.0002)
  Label smoothing: 0.1

Loss Weights:
  Reconstruction: 0.999
  Confidence: 0.0005
  Localization: 0
  IoU: 0.0005

Epoch 1/10: 100% 782/782 [00:28<00:00, 27.22it/s, G=0.0305, D=0.9560, IoU=0.703, D_upd=✓]

Epoch 1 Summary:
  Generator Loss: 0.0364
  Discriminator Loss: 0.5690
  Reconstruction Loss: 0.0357
  IoU: 0.304
  D Accuracy: 0.94
  D Updates: 391/782
Saved checkpoint to ./localizer/checkpoints/best_model.pth
  New best model saved (loss: 0.0357)
Epoch 2/10: 100% 782/782 [00:26<00:00, 29.90it/s, G=0.0380, D=0.8578, IoU=0.669, D_upd=✓]

Epoch 2 Summary:
  Generator Loss: 0.0291
  Discriminator Loss: 0.4630
  Reconstruction Loss: 0.0282
  IoU: 0.344
  D Accuracy: 0.98
  D Updates: 391/782
Saved checkpoint to ./localizer/check

In [ ]:
!python staged/train_staged.py --split_seed 42 \
  --d_update_ratio 2 \
  --checkpoint_dir ./staged/checkpoints \
  --dataset cifar10 \
  --num_epochs 30

Split info saved to ./staged/checkpoints/split_info.json

STAGED CONTEXT ENCODER TRAINING
Starting from epoch 0
Dataset: cifar10
Train batches: 782, Val batches: 157
Device: cuda
Mask: random_square (64x64)
Split seed: 42 (same seed = same split across models)
Fixed validation images: 10

--- Staged Training Schedule ---
Stage 1 (Binary D): Epochs 0-19
Transition: Epochs 20-24
Stage 2 (Localizer D): Epochs 20+

--- Stage 1 Parameters (Binary Discriminator) ---
  Lambda Rec: 0.999
  Lambda Adv: 0.001

--- Stage 2 Parameters (Localizer Discriminator) ---
  Capacity factor: 1.0
  Dropout rate: 0.0
  Num layers: 5
  Lambda Conf: 0.0005
  Lambda Loc: 0.0001 (NEGATIVE in G loss)
  Lambda IoU: 0.0005
  D Update Ratio: 1/2

Epoch 1/30 [Stage 1]: 100% 782/782 [01:11<00:00, 10.86it/s, G=0.0381, D=0.0448, Rec=0.0336]

  Running validation...
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed

In [ ]:
!python evaluate_models.py \
    --l2_checkpoint ./baseline/checkpoints_l2/best_model.pth \
    --baseline_checkpoint ./baseline/checkpoints_pathak/best_model.pth \
    --localizer_checkpoint ./localizer/checkpoints/best_model.pth \
    --staged_checkpoint ./staged/checkpoints/best_model.pth \
    --dataset celeba \
    --eval_set test \
    --split_seed 42 \
    --mask_seed 456 \
    --output_dir ./evaluation_results

Using device: cuda

Loading dataset with split_seed=42...
Loaded images from ./data/celeba (flat folder)
Total images found: 30113
Dataset splits (seed=42):
  Train: 24,090 samples (80.0%)
  Val:   3,011 samples (10.0%)
  Test:  3,012 samples (10.0%)
Evaluating on TEST set (3012 images)

Loading models...
Loaded checkpoint from epoch 29
  ✓ Loaded l2_only from ./baseline/checkpoints_l2/best_model.pth
Loaded checkpoint from epoch 28
  ✓ Loaded baseline from ./baseline/checkpoints_pathak/best_model.pth
Loaded checkpoint from epoch 29
  ✓ Loaded localizer from ./localizer/checkpoints/best_model.pth
Loaded checkpoint from epoch 29 (Stage 2)
  ✓ Loaded staged from ./staged/checkpoints/best_model.pth

Evaluating models (mask_seed=456 for reproducibility)...
Mask type: random_square, Mask size: 64x64

Evaluating: L2 Only
Evaluating:   0% 0/95 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.

In [ ]:
!python test_discriminator_detection.py \
    --checkpoint ./staged/checkpoints/best_model.pth \
    --model_type staged \
    --output_dir ./diagnostic_staged

Loading model from ./staged/checkpoints/best_model.pth...
Loaded checkpoint from epoch 29 (Stage 2)
  Staged model loaded - Current stage: 2
  Transition epoch: 20
  Testing BOTH discriminators (binary + localizer)
Loading CIFAR-10 train set...

Test 1: Real images (unmodified)
  [Localizer] Confidence: 0.5275 ± 0.1932
  (Should be LOW ~0 for real images)
  [Binary D]  Confidence: 0.2821 ± 0.2193

Test 2: Gaussian blur regions (NO boundary discontinuity)

  Sigma = 3.0:
    [Localizer] Confidence: 0.5354, IoU: 0.4367 ± 0.1420
    [Binary D]  Confidence: 0.2886
Saved: ./diagnostic_staged/blur_sigma_3.0.png

  Sigma = 5.0:
    [Localizer] Confidence: 0.5455, IoU: 0.4250 ± 0.1825
    [Binary D]  Confidence: 0.2525
Saved: ./diagnostic_staged/blur_sigma_5.0.png

  Sigma = 10.0:
    [Localizer] Confidence: 0.5549, IoU: 0.4364 ± 0.1750
    [Binary D]  Confidence: 0.1959
Saved: ./diagnostic_staged/blur_sigma_10.0.png

  Sigma = 20.0:
    [Localizer] Confidence: 0.5733, IoU: 0.4949 ± 0.1629
   

In [ ]:
!python compare_models.py \
    --l2_checkpoint ./baseline/checkpoints_l2/best_model.pth \
    --baseline_checkpoint ./baseline/checkpoints_pathak/best_model.pth \
    --localizer_checkpoint ./localizer/checkpoints/best_model.pth \
    --staged_checkpoint ./staged/checkpoints/best_model.pth \
    --output_dir ./comparison_results \
    --mask_size 64 \
    --image_seed 42

Using device: cuda

Loading CIFAR-10 dataset...

Selecting one image per class (seed=42)...
Selected images:
  Class 0 (airplane): index 989
  Class 1 (automobile): index 4401
  Class 2 (bird): index 8631
  Class 3 (cat): index 2795
  Class 4 (deer): index 1233
  Class 5 (dog): index 729
  Class 6 (frog): index 7025
  Class 7 (horse): index 210
  Class 8 (ship): index 6366
  Class 9 (truck): index 1175

Generating masks (seed=123)...

Loading models...
  Loading L2-only model...
Loaded checkpoint from epoch 29
  Loaded l2_only model from ./baseline/checkpoints_l2/best_model.pth
  Loading baseline model...
Loaded checkpoint from epoch 28
  Loaded baseline model from ./baseline/checkpoints_pathak/best_model.pth
  Loading localizer model...
Loaded checkpoint from epoch 29
  Loaded localizer model from ./localizer/checkpoints/best_model.pth
  Loading staged model...
Loaded checkpoint from epoch 29 (Stage 2)
  Loaded staged model from ./staged/checkpoints/best_model.pth

Generating completi